# 📊 ANN Regression Model for PCF-SPR Sensor (Final Version)

Now that we successfully predicted **Loss** alone, we extend our ANN to predict **both Loss and Sensitivity** at once.

- **Inputs:** wavelength (w), refractive index (na), gold radius (rg), imag(neff)  
- **Outputs:** confinement loss, sensitivity (SA/RIU)  
- **Architecture:** 3 hidden layers × 32 neurons, ReLU activations  
- **Optimizer:** Adam with gradient clipping  
- **Loss function:** Mean Squared Error (MSE)  

---


In [16]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [17]:
# Step 2: Load dataset
data = pd.read_csv("Collected_Data.csv")
data.columns = data.columns.str.strip()  # clean column names

print(data.head())


      w    na                                       neff  re(neff)  \
0  0.60  1.31   1.4519323887669042-3.291653714847547E-6i  1.451932   
1  0.62  1.31     1.45091066181998-3.408409361911753E-6i  1.450911   
2  0.64  1.31   1.4499279284661912-4.654727785503613E-6i  1.449928   
3  0.66  1.31   1.4489779158457035-8.768782197488685E-6i  1.448978   
4  0.68  1.31  1.4480534509365457-2.0683030246449427E-5i  1.448053   

   imag(neff)       loss  SA(1/RIU)  rg(um)  
0    0.000003   3.056809   4.515266     0.3  
1    0.000003   3.063131 -12.868908     0.3  
2    0.000005   4.052470  19.683089     0.3  
3    0.000009   7.402882  36.599197     0.3  
4    0.000021  16.947698  48.733377     0.3  


---
## Step 3: Define Inputs and Outputs
- Inputs (X): `w`, `na`, `rg(um)`, `imag(neff)`
- Outputs (y): `loss`, `SA(1/RIU)`
---


In [18]:
X = data[['w', 'na', 'rg(um)', 'imag(neff)']].values
y = data[['loss', 'SA(1/RIU)']].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use RobustScaler for stability (handles outliers)
scaler_X = RobustScaler()
scaler_y = RobustScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)


---
## Step 4: Build the ANN
- 3 hidden layers × 32 neurons  
- ReLU activation  
- Adam optimizer (small learning rate, with gradient clipping)  
- Output layer: 2 neurons → predicts `[loss, sensitivity]`  
---


In [19]:
# Build ANN
model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'),
    Dense(32, activation='relu', kernel_initializer='he_normal'),
    Dense(32, activation='relu', kernel_initializer='he_normal'),
    Dense(2)  # two outputs now
])

opt = Adam(learning_rate=0.0001, clipnorm=1.0)
model.compile(optimizer=opt, loss='mse')


c:\Users\dskum\Python\deeplearning\langchain\venv\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
print(data[['loss', 'SA(1/RIU)']].describe())
print(data[['loss', 'SA(1/RIU)']].head(20))



              loss    SA(1/RIU)
count  1640.000000  1476.000000
mean     33.936850   -52.470045
std      70.336385   134.193606
min       0.652363  -933.425313
25%       5.324138   -65.704251
50%       9.745017   -11.463914
75%      22.962796    11.125778
max     550.855615    90.047087
         loss   SA(1/RIU)
0    3.056809    4.515266
1    3.063131  -12.868908
2    4.052470   19.683089
3    7.402882   36.599197
4   16.947698   48.733377
5   50.141533   62.313839
6   56.197434    4.225553
7   23.525991 -239.244891
8   12.244001 -185.325562
9    7.916966 -122.660728
10   5.889681  -88.018715
11   4.809559  -67.160251
12   4.191613  -53.304027
13   3.829753  -43.397138
14   3.624729  -35.928141
15   3.524091  -30.074831
16   3.498119  -25.353017
17   3.529067  -21.458331
18   3.605924  -18.188924
19   3.721673  -15.404936


In [21]:
print(np.any(np.isnan(y)))
print(np.any(np.isinf(y)))


True
False


In [22]:
data['loss'] = np.log1p(data['loss'])   # log(1+x) to keep positive


In [23]:
data['SA(1/RIU)'] = data['SA(1/RIU)'].clip(-1000, 1000)


In [24]:
print(data[['w','na','rg(um)','imag(neff)']].describe())


                 w           na       rg(um)    imag(neff)
count  1640.000000  1640.000000  1640.000000  1.640000e+03
mean      1.000000     1.355000     0.450000  6.698639e-05
std       0.236715     0.028732     0.111838  1.485154e-04
min       0.600000     1.310000     0.300000  7.020000e-07
25%       0.800000     1.330000     0.375000  8.830000e-06
50%       1.000000     1.355000     0.450000  1.910000e-05
75%       1.200000     1.380000     0.525000  4.262500e-05
max       1.400000     1.400000     0.600000  1.223253e-03


In [25]:
data = data.dropna()


In [26]:
print(data.isnull().sum())


w             0
na            0
neff          0
re(neff)      0
imag(neff)    0
loss          0
SA(1/RIU)     0
rg(um)        0
dtype: int64


In [27]:
data

,w,na,neff,re(neff),imag(neff),loss,SA(1/RIU),rg(um)
0,0.60,1.31,1.4519323887669042-3.291653714847547E-6i,1.451932,0.000003,1.400397,4.515266,0.3
1,0.62,1.31,1.45091066181998-3.408409361911753E-6i,1.450911,0.000003,1.401954,-12.868908,0.3
2,0.64,1.31,1.4499279284661912-4.654727785503613E-6i,1.449928,0.000005,1.619877,19.683089,0.3
3,0.66,1.31,1.4489779158457035-8.768782197488685E-6i,1.448978,0.000009,2.128575,36.599197,0.3
4,0.68,1.31,1.4480534509365457-2.0683030246449427E-5i,1.448053,0.000021,2.887462,48.733377,0.3
...,...,...,...,...,...,...,...,...
1594,1.32,1.39,1.4229503202450096-7.410097895945923E-5i,1.422950,0.000074,3.474423,-481.993369,0.6
1595,1.34,1.39,1.4221526626172099-7.32035883551411E-5i,1.422153,0.000073,3.448055,-481.988075,0.6
1596,1.36,1.39,1.4213535810163063-7.299525179100927E-5i,1.421354,0.000073,3.430957,-491.971025,0.6
1597,1.38,1.39,1.4205530013982923-7.33873498248663E-5i,1.420553,0.000073,3.422015,-509.253686,0.6


In [28]:
# Step 5: Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16,
                    validation_split=0.2, verbose=1)


Epoch 1/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: nan - val_loss: nan
Epoch 2/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - val_loss: nan
Epoch 4/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - val_loss: nan
Epoch 7/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - val_loss: nan
Epoch 8/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - val_loss: nan
Epoch 9/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - val_loss: nan
Epoch 11/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 12/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: nan - val_loss: nan
Epoch 13/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/st

KeyboardInterrupt: 